In [2]:

import pandas as pd

In [3]:
df = pd.read_csv('./data/survey_results_public.csv')
df_schema = pd.read_csv('./data/survey_results_schema.csv')

In [7]:
country_series = df['Country']

In [10]:
country_series[country_series == 'Iran, Islamic Republic of...']

74       Iran, Islamic Republic of...
87       Iran, Islamic Republic of...
142      Iran, Islamic Republic of...
176      Iran, Islamic Republic of...
227      Iran, Islamic Republic of...
                     ...             
88568    Iran, Islamic Republic of...
88635    Iran, Islamic Republic of...
88929    Iran, Islamic Republic of...
88938    Iran, Islamic Republic of...
89183    Iran, Islamic Republic of...
Name: Country, Length: 577, dtype: object